# משימת סיווג טקסט

כפי שציינו, נתמקד במשימת סיווג טקסט פשוטה המבוססת על **AG_NEWS** dataset, שמטרתה לסווג כותרות חדשות לאחת מתוך 4 קטגוריות: עולם, ספורט, עסקים ומדע/טכנולוגיה.

## מערך הנתונים

מערך נתונים זה מובנה בתוך המודול [`torchtext`](https://github.com/pytorch/text), כך שניתן לגשת אליו בקלות.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

כאן, `train_dataset` ו-`test_dataset` מכילים אוספים שמחזירים זוגות של תווית (מספר של מחלקה) וטקסט בהתאמה, לדוגמה:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

אז, בואו נדפיס את עשרת הכותרות החדשות הראשונות מהמאגר שלנו:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

מכיוון שמערכי נתונים הם איטרטורים, אם אנחנו רוצים להשתמש בנתונים מספר פעמים אנחנו צריכים להמיר אותם לרשימה:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## טוקניזציה

עכשיו אנחנו צריכים להמיר טקסט ל**מספרים** שניתן לייצג כטנסורים. אם אנחנו רוצים ייצוג ברמת מילים, יש לבצע שני דברים:
* להשתמש ב**טוקנייזר** כדי לחלק את הטקסט ל**טוקנים**
* לבנות **אוצר מילים** של אותם טוקנים.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

באמצעות אוצר מילים, אנו יכולים בקלות לקודד את המחרוזת המפוצלת שלנו למערך של מספרים:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## ייצוג טקסט בשיטת Bag of Words

מכיוון שמילים מייצגות משמעות, לפעמים אפשר להבין את המשמעות של טקסט רק על ידי התבוננות במילים הבודדות, בלי להתחשב בסדר שלהן במשפט. לדוגמה, כאשר מסווגים חדשות, מילים כמו *מזג אוויר*, *שלג* עשויות להצביע על *תחזית מזג אוויר*, בעוד שמילים כמו *מניות*, *דולר* ייחשבו כחלק מ*חדשות כלכליות*.

ייצוג וקטורי **Bag of Words** (BoW) הוא הייצוג הווקטורי המסורתי הנפוץ ביותר. כל מילה מקושרת לאינדקס בווקטור, והאלמנט בווקטור מכיל את מספר ההופעות של מילה מסוימת במסמך נתון.

![תמונה שמראה כיצד ייצוג וקטורי בשיטת Bag of Words מיוצג בזיכרון.](../../../../../translated_images/he/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: ניתן גם לחשוב על BoW כסכום של כל הווקטורים המקודדים בשיטת one-hot עבור המילים הבודדות בטקסט.

להלן דוגמה ליצירת ייצוג בשיטת Bag of Words באמצעות ספריית Scikit Learn בשפת Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

כדי לחשב וקטור של שקית מילים מתוך הייצוג הווקטורי של מערך הנתונים AG_NEWS שלנו, נוכל להשתמש בפונקציה הבאה:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **הערה:** כאן אנו משתמשים במשתנה הגלובלי `vocab_size` כדי לציין את גודל ברירת המחדל של אוצר המילים. מכיוון שלעתים קרובות גודל אוצר המילים די גדול, ניתן להגביל את גודל אוצר המילים למילים הנפוצות ביותר. נסו להקטין את הערך של `vocab_size` ולהריץ את הקוד למטה, ותראו כיצד זה משפיע על הדיוק. אתם אמורים לצפות לירידה מסוימת בדיוק, אך לא דרמטית, בתמורה לביצועים גבוהים יותר.


## אימון מסווג BoW

עכשיו, כשכבר למדנו איך לבנות ייצוג Bag-of-Words לטקסט שלנו, בואו נאמן מסווג על בסיסו. קודם כל, עלינו להמיר את מערך הנתונים שלנו לאימון כך שכל הייצוגים הווקטוריים הפוזיציוניים יומרו לייצוג Bag-of-Words. ניתן לעשות זאת על ידי העברת הפונקציה `bowify` כפרמטר `collate_fn` ל-`DataLoader` הסטנדרטי של torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

עכשיו נגדיר רשת עצבית מסווגת פשוטה שמכילה שכבה ליניארית אחת. גודל וקטור הקלט שווה ל-`vocab_size`, וגודל הפלט מתאים למספר הקטגוריות (4). מכיוון שאנחנו פותרים משימת סיווג, פונקציית ההפעלה הסופית היא `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

עכשיו נגדיר לולאת אימון סטנדרטית של PyTorch. מכיוון שהמערך שלנו די גדול, למטרת ההוראה שלנו נאמן רק למשך אפוקה אחת, ולפעמים אפילו פחות מאפוקה (הגדרת הפרמטר `epoch_size` מאפשרת לנו להגביל את האימון). כמו כן, נדווח על דיוק האימון המצטבר במהלך האימון; תדירות הדיווח מוגדרת באמצעות הפרמטר `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## בי-גרמים, טרי-גרמים ו-N-גרמים

אחת המגבלות של גישת שקית המילים היא שחלק מהמילים הן חלק מביטויים רב-מילתיים. לדוגמה, המילה 'נקניקייה' ('hot dog') בעלת משמעות שונה לחלוטין מהמילים 'חם' ('hot') ו'כלב' ('dog') בהקשרים אחרים. אם נייצג תמיד את המילים 'חם' ו'כלב' באמצעות אותם וקטורים, זה עלול לבלבל את המודל שלנו.

כדי להתמודד עם זה, **ייצוגי N-גרמים** משמשים לעיתים קרובות בשיטות לסיווג מסמכים, שבהן התדירות של כל מילה, זוג מילים או שלישיית מילים מהווה תכונה שימושית לאימון מסווגים. בייצוג בי-גרמים, לדוגמה, נוסיף את כל זוגות המילים לאוצר המילים, בנוסף למילים המקוריות.

להלן דוגמה כיצד ליצור ייצוג שקית מילים של בי-גרמים באמצעות Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

החיסרון העיקרי בגישת N-gram הוא שהגודל של אוצר המילים מתחיל לגדול בצורה מהירה מאוד. בפועל, אנחנו צריכים לשלב את ייצוג ה-N-gram עם טכניקות לצמצום ממדים, כמו *embeddings*, עליהן נדון ביחידה הבאה.

כדי להשתמש בייצוג N-gram במאגר הנתונים **AG News**, אנחנו צריכים לבנות אוצר מילים מיוחד של ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


נוכל להשתמש באותו קוד כמו למעלה כדי לאמן את הסיווג, אך זה יהיה מאוד לא יעיל מבחינת זיכרון. ביחידה הבאה, נאמן סיווג ביגרם באמצעות embeddings.

> **הערה:** ניתן להשאיר רק את ה-ngrams שמופיעים בטקסט יותר ממספר הפעמים שצוין. זה יבטיח שביגרמים נדירים יושמטו, ויקטין משמעותית את הממדיות. כדי לעשות זאת, יש להגדיר את הפרמטר `min_freq` לערך גבוה יותר, ולצפות בשינוי באורך אוצר המילים.


## תדירות מונח-היפוך תדירות מסמך TF-IDF

בייצוג BoW, הופעות של מילים מקבלות משקל שווה, ללא קשר למילה עצמה. עם זאת, ברור שמילים נפוצות כמו *a*, *in* וכו' הן הרבה פחות חשובות לסיווג מאשר מונחים מיוחדים. למעשה, ברוב משימות עיבוד שפה טבעית, יש מילים שהן רלוונטיות יותר מאחרות.

**TF-IDF** מייצג **תדירות מונח–היפוך תדירות מסמך**. זהו וריאציה של תיקיית מילים (bag of words), שבה במקום ערך בינארי 0/1 שמציין את הופעתה של מילה במסמך, נעשה שימוש בערך עשרוני, הקשור לתדירות הופעת המילה בקורפוס.

באופן פורמלי יותר, המשקל $w_{ij}$ של מילה $i$ במסמך $j$ מוגדר כ:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
כאשר
* $tf_{ij}$ הוא מספר ההופעות של $i$ ב-$j$, כלומר ערך BoW שראינו קודם
* $N$ הוא מספר המסמכים באוסף
* $df_i$ הוא מספר המסמכים המכילים את המילה $i$ בכל האוסף

ערך TF-IDF $w_{ij}$ עולה באופן יחסי למספר הפעמים שמילה מופיעה במסמך ומותאם לפי מספר המסמכים בקורפוס שמכילים את המילה, מה שעוזר להתחשב בעובדה שיש מילים שמופיעות בתדירות גבוהה יותר מאחרות. לדוגמה, אם מילה מופיעה *בכל* המסמכים באוסף, $df_i=N$, ו-$w_{ij}=0$, והמונחים הללו יוזנחו לחלוטין.

ניתן ליצור בקלות וקטוריזציה של טקסט באמצעות TF-IDF בעזרת Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## מסקנה

עם זאת, למרות שייצוגי TF-IDF מספקים משקל תדירות למילים שונות, הם אינם מסוגלים לייצג משמעות או סדר. כפי שאמר הבלשן המפורסם ג'יי. אר. פירת' בשנת 1935, "המשמעות המלאה של מילה תמיד תלויה בהקשר, ואין לקחת ברצינות שום מחקר על משמעות מחוץ להקשר." בהמשך הקורס נלמד כיצד ללכוד מידע הקשרי מטקסט באמצעות מודלים לשוניים.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
